**Notebook summary:** This notebook takes the existing capacity from Markal Spreadsheets and converts them to OEO friendly database fromat. It splits the existing capacity given in Markal (segregated by census divisions) and downscales this to state level estimates using census population data. These state level existing capacities are then aggregated back up to the OEO regions.

# Essentials

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3
import urllib.request
import os
import seaborn as sns
import gzip
from pandas.api.types import CategoricalDtype
import zipfile
sns.set()

In [3]:
region_names = dict()
region_names[1] = 'CA'
region_names[2] = 'NW'
region_names[3] = 'SW'
region_names[4] = 'TX'
region_names[5] = 'CEN'
region_names[6] = 'N_CEN'
region_names[7] = 'SE'
region_names[8] = 'MID_AT'
region_names[9] = 'NE'

In [4]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None and len(lst_cols) > 0
            and not isinstance(lst_cols,
                               (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame(
        {col: np.repeat(df[col].values, lens)
         for col in idx_cols}, index=idx).assign(
             **{
                 col: np.concatenate(df.loc[lens > 0, col].values)
                 for col in lst_cols
             }))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens == 0, idx_cols],
                          sort=False).fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:
        res = res.reset_index(drop=True)
    return res

In [5]:
def reverse_appending_keys(dictionary, key, value):
    if value not in dictionary:
        dictionary[value] = key
    elif type(dictionary[value]) == list:
        dictionary[value].append(key)
    else:
        dictionary[value] = [dictionary[value], key]

# Read Data Files

In [6]:
df_commercial = pd.read_excel('./rawData/commercial_existingCapacity_byState.ods', engine="odf")
df_residential = pd.read_excel('./rawData/residential_existingCapacity_byState.ods', engine="odf")
df_pop = pd.read_excel('./rawData/STATE_populations_CENSUS.xlsx')
state_groupings = pd.read_csv('./rawData/state_regions_cluster.csv')

# Getting Fractional Population Data From Census

In [7]:
df_pop['State'] = df_pop.State.str.strip('.')

In [8]:
df_pop_test = df_pop[["CENSUS_DIV", *tuple(range(2010, 2020))]]
df_temp = df_pop_test.groupby(['CENSUS_DIV']).apply(lambda x: x / x.sum())
df_pop.update(df_temp)

In [8]:
census_dict = df_pop.set_index('STUSPS').to_dict()['CENSUS_DIV']

In [9]:
census_dict_reversed = dict()
for key, val in census_dict.items():
    reverse_appending_keys(census_dict_reversed, key, val)

census_dict_reversed = {
    key: str(value)
    for (key, value) in census_dict_reversed.items()
}

# Updating Commercial Existing Capacity

In [10]:
# Split the existing capacity equally amongst different versions of the same technology
df_comm_melt = pd.melt(df_commercial,
                       id_vars=[
                           "Sector", "Subsector", "Tech_Code_Pandas",
                           "Tech Description", "tech", "vintage",
                           "Tech From Markal", "Notes"
                       ],
                       var_name=["CENSUS_DIV"],
                       value_name="Value")
df_comm_melt['tech_count'] = df_comm_melt.groupby(
    ['Tech_Code_Pandas', 'CENSUS_DIV'])['Value'].transform('count')
df_comm_melt['Value'] = df_comm_melt['Value'] / df_comm_melt['tech_count']

# replacing census division with US states
df_comm_melt = df_comm_melt.replace({"CENSUS_DIV": census_dict_reversed})
df_comm_melt['CENSUS_DIV'] = df_comm_melt['CENSUS_DIV'].str.strip("[]")
df_comm_melt['CENSUS_DIV'] = df_comm_melt['CENSUS_DIV'].replace({'\'': ''},
                                                                regex=True)

In [11]:
df_comm_explod = explode(
    df_comm_melt.assign(CENSUS_DIV=df_comm_melt.CENSUS_DIV.str.split(',')),
    'CENSUS_DIV',
    preserve_index=True).reset_index(drop=True)
df_comm_explod['CENSUS_DIV'] = df_comm_explod.CENSUS_DIV.str.replace(' ', '')
df_pop = df_pop.drop(['State', 'CENSUS_DIV', *tuple(range(2011, 2020))],
                     axis=1)
df_comm_pop = df_comm_explod.merge(df_pop,
                                   left_on='CENSUS_DIV',
                                   right_on='STUSPS',
                                   how='inner',
                                   validate="many_to_one")
df_comm_pop['ExistingCap'] = df_comm_pop['Value'] * df_comm_pop[2010]
df_comm_pop = df_comm_pop.drop(['STUSPS', 'Value'], axis=1)

state_groupings['NAME'] = state_groupings['NAME'].str.upper()
df_comm_OEO = df_comm_pop.merge(state_groupings,
                                left_on='CENSUS_DIV',
                                right_on='STUSPS')

In [12]:
df_comm_OEO['Region_OEO'] = df_comm_OEO['Region'].map(region_names)
df_comm_OEO = df_comm_OEO.drop(
    ['CENSUS_DIV', 'NAME', 'STUSPS', 'Region', 'Notes'], axis=1)

In [13]:
df_comm_OEO_1 = df_comm_OEO.groupby([
    'Sector', 'Subsector', 'Tech Description', 'Tech_Code_Pandas', 'tech',
    'vintage', 'Region_OEO'
])['ExistingCap'].sum().reset_index()
# df_comm_OEO_1 = df_comm_OEO.groupby(['Sector', 'Subsector', 'Tech Description', 'Tech From Markal', 'Tech_Code_Pandas', 'tech', 'vintage', 'Region_OEO'])['ExistingCap'].sum().reset_index()
# df_comm_OEO_2 = df_comm_OEO.groupby(['tech', 'vintage', 'Region_OEO'])['ExistingCap'].sum().reset_index()

# Updating Residential Existing Capacity

In [14]:
df_res_melt = pd.melt(df_residential,
                      id_vars=[
                          "Sector", "Subsector", "Tech_Code_Pandas",
                          "Tech Description", "tech", "vintage",
                          "Tech From Markal", "Notes"
                      ],
                      var_name=["CENSUS_DIV"],
                      value_name="Value")
df_res_melt['tech_count'] = df_res_melt.groupby(
    ['Tech_Code_Pandas', 'CENSUS_DIV'])['Value'].transform('count')
df_res_melt['Value'] = df_res_melt['Value'] / df_res_melt['tech_count']

df_res_melt = df_res_melt.replace({"CENSUS_DIV": census_dict_reversed})
df_res_melt['CENSUS_DIV'] = df_res_melt['CENSUS_DIV'].str.strip("[]")
df_res_melt['CENSUS_DIV'] = df_res_melt['CENSUS_DIV'].replace({'\'': ''},
                                                              regex=True)

In [15]:
df_res_explod = explode(
    df_res_melt.assign(CENSUS_DIV=df_res_melt.CENSUS_DIV.str.split(',')),
    'CENSUS_DIV',
    preserve_index=True).reset_index(drop=True)
df_res_explod['CENSUS_DIV'] = df_res_explod.CENSUS_DIV.str.replace(' ', '')
df_res_pop = df_res_explod.merge(df_pop,
                                 left_on='CENSUS_DIV',
                                 right_on='STUSPS',
                                 how='inner',
                                 validate="many_to_one")
df_res_pop['ExistingCap'] = df_res_pop['Value'] * df_res_pop[2010]
df_res_pop = df_res_pop.drop(['STUSPS', 'Value'], axis=1)
df_res_OEO = df_res_pop.merge(state_groupings,
                              left_on='CENSUS_DIV',
                              right_on='STUSPS')

In [16]:
df_res_OEO['Region_OEO'] = df_res_OEO['Region'].map(region_names)
df_res_OEO = df_res_OEO.drop(
    ['CENSUS_DIV', 'NAME', 'STUSPS', 'Region', 'Notes'], axis=1)

In [17]:
df_res_OEO_1 = df_res_OEO.groupby(['Sector', 'Subsector', 'Tech Description', 'Tech_Code_Pandas', 'tech', 'vintage', 'Region_OEO'])['ExistingCap'].sum().reset_index()
# df_res_OEO_2 = df_res_OEO.groupby(['tech', 'vintage', 'Region_OEO'])['ExistingCap'].sum().reset_index()
# df_res_OEO_2 = df_res_OEO.groupby(['tech', 'vintage', 'Region_OEO', 'Tech From Markal'])['ExistingCap'].sum().reset_index()

# Output Files

In [20]:
df_res_OEO_1.rename(columns = {"Region_OEO":"regions", "ExistingCap": "exist_cap"}, inplace = True)
df_res_OEO_1['exist_cap_units'] = 'PJ'
df_comm_OEO_1.rename(columns = {"Region_OEO":"regions", "ExistingCap": "exist_cap"}, inplace = True)
df_comm_OEO_1['exist_cap_units'] = 'PJ'

In [21]:
df_res_OEO_1[['regions', 'tech', 'vintage', 'exist_cap', 'exist_cap_units']].to_csv('./TablesForDB/residential_existingCapacity.csv')
df_comm_OEO_1[['regions', 'tech', 'vintage', 'exist_cap', 'exist_cap_units']].to_csv('./TablesForDB/commercial_existingCapacity.csv')

# Diagnostic bits of code

In [ ]:
df_res_OEO_1.loc[df_res_OEO_1['Subsector'] == 'Residential Space Heating']['ExistingCap'].agg(['sum', 'mean', 'min', 'max'])

In [665]:
# list_low = list(df_res_OEO_1.tech.unique())
# list_high = list(df_res_OEO_2.tech.unique())
# list(set(list_low) - set(list_high))
# df_res_OEO_2.loc[df_res_OEO_2['tech'].isin(list(set(list_high) - set(list_low))), ]

,tech,vintage,Region_OEO,ExistingCap
117,R_LT_HIRST_ROL_R,2015,CA,0.0
118,R_LT_HIRST_ROL_R,2015,CEN,0.0
119,R_LT_HIRST_ROL_R,2015,MID_AT,0.0
120,R_LT_HIRST_ROL_R,2015,NE,0.0
121,R_LT_HIRST_ROL_R,2015,NW,0.0
...,...,...,...,...
1111,R_WH_HP_ELC_R,2015,NW,0.0
1112,R_WH_HP_ELC_R,2015,N_CEN,0.0
1113,R_WH_HP_ELC_R,2015,SE,0.0
1114,R_WH_HP_ELC_R,2015,SW,0.0


In [ ]:
store1 = df_res_OEO.groupby(['Sector', 'Subsector', 'tech', 'vintage', 'Region_OEO'])['ExistingCap'].sum().reset_index()
store2 = df_res_OEO.groupby(['Sector', 'Subsector', 'tech', 'Tech From Markal', 'vintage', 'Region_OEO'])['ExistingCap'].sum().reset_index()
store2 = store2[['Sector', 'Subsector', 'tech', 'vintage', 'Region_OEO', 'ExistingCap']]
# store1.merge(store2, how='inner', indicator=False)
store1.merge(store2, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']['tech'].unique()